# PRÁCTICA 4

- JORGE CASAN VÁZQUEZ

Nos importamos las librerías necesarias

In [18]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import PlaintextCorpusReader
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import pyLDAvis.gensim
import nltk, re, pprint, spacy
import textacy
from langdetect import *
from nltk.stem import WordNetLemmatizer
import pandas as pd

Ahora creamos el conjunto de documentos, los cuales serán un total de 10 documentos, los cuales están en castellano y cubren diferentes tópicos cada uno de ellos 

# Importamos todos nuestros documentos guardados 

In [19]:
doc_list=[]
docs_dir = "C:/Users/jcasa/OneDrive/Escritorio/2º SEMESTRE/Analisis de informacion no estructurada/Actividad_4/Corpus Documentos"
documentos = PlaintextCorpusReader(docs_dir, '.*')

for fileid in documentos.fileids():
    doc_list.append(documentos.raw(fileids=fileid))
documentos.fileids()

['Documento1',
 'Documento10',
 'Documento2',
 'Documento3',
 'Documento4',
 'Documento5',
 'Documento6',
 'Documento7',
 'Documento8',
 'Documento9']

# Análisis completo de los documentos

Realizamos un análisis sobre los siguientes términos, sobre el primer documento:
- Frases
- Palabras
- POS
- Forma Normal
- Entidades

In [36]:
documento = documentos.raw(fileids="Documento1")
documento

'La historia de la ciudad de Valencia abarca desde la época anterior a la fundación formal de la ciudad cuando la región ya estaba habitada por pueblos prerromanos hasta la actualidad. Gracias a la fértil huerta sobre la que se sustenta, los alrededores de Valencia (España) fueron ya una zona de alta densidad humana antes de su fundación en el siglo II a. C. por los romanos. Desde entonces, la ciudad ha constituido uno de los mayores núcleos urbanos de la península ibérica'

In [37]:
nlp = spacy.load('es_core_news_sm')
doc = nlp(documento)

print("\Frases")
for sent in doc.sents:
    print(sent.text)

print("\nTokens")
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)

print("\nEntidades")
for ent in doc.ents:
    print(ent.text, ent.label_)

<input>:4: DeprecationWarning: invalid escape sequence \F
<input>:4: DeprecationWarning: invalid escape sequence \F
<input>:4: DeprecationWarning: invalid escape sequence \F
<ipython-input-37-742bfb4a5e78>:4: DeprecationWarning: invalid escape sequence \F
  print("\Frases")


\Frases
La historia de la ciudad de Valencia abarca desde la época anterior a la fundación formal de la ciudad cuando la región ya estaba habitada por pueblos prerromanos hasta la actualidad.
Gracias a la fértil huerta sobre la que se sustenta, los alrededores de Valencia (España) fueron ya una zona de alta densidad humana antes de su fundación en el siglo II a. C. por los romanos.
Desde entonces, la ciudad ha constituido uno de los mayores núcleos urbanos de la península ibérica

Tokens
La La DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
historia historia NOUN NOUN__Gender=Fem|Number=Sing nsubj
de de ADP ADP__AdpType=Prep case
la lo DET DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art det
ciudad ciudad NOUN NOUN__Gender=Fem|Number=Sing nmod
de de ADP ADP__AdpType=Prep case
Valencia Valencia PROPN PROPN___ nmod
abarca abarcar VERB VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin ROOT
desde desde ADP ADP__AdpType=Prep case
la lo DET DET__Definite=Def|Gend

Spacy ofrecía mejores resultados que la librería NLTK para trabajar sobre textos en español, por lo que decidimos trabajar sobre ella. Cargamos el modelo de español sobre Spacy y vamos a cargar la lista de palabras vacías en español

# Preparación previa para Análisis LDA

Pasaremos todas las palabras a minúsculas y a las formas normales y retiraremos las palabras vacías

In [38]:
es_stop = get_stop_words('es')

In [39]:
textos = []
palabras = []

for doc in doc_list:
    doc_min = doc.lower()
    doc = nlp(doc_min)
    
    for token in doc:
        if (not token.lemma_ in es_stop) and (len(token.lemma_)>1):
            palabras.append(token.lemma_)
    
    textos.append(palabras)
    palabras = []

# Construimos el diccionario y la matriz de términos-documentos

In [40]:
# Construímos el diccionario, que es una lista de las palabras que aparecen en el corpus textos

dictionary = corpora.Dictionary(textos)
    
# Ahora generamos la matriz términos documentos que nos hace falta para llamar a la función que genera el modelo LDA
corpus = [dictionary.doc2bow(texto) for texto in textos]

# Modelo LDA

In [ ]:
vldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=7, id2word = dictionary, passes=50)

# Representar los valores obtenidos

In [41]:
print(ldamodel.print_topics(num_topics=7, num_words=10))

[(0, '0.034*"ciudad" + 0.027*"ser" + 0.014*"sobrar" + 0.014*"haber" + 0.014*"cordillera" + 0.014*"poblar" + 0.014*"sureste" + 0.014*"fundación" + 0.014*"valencia" + 0.007*"romano"'), (1, '0.030*"abogar" + 0.020*"unir" + 0.020*"seriar" + 0.020*"mike" + 0.011*"sobrar" + 0.011*"después" + 0.011*"alejar" + 0.011*"gracia" + 0.011*"historia" + 0.011*"gran"'), (2, '0.049*"dato" + 0.022*"comer" + 0.022*"fuente" + 0.022*"ciencia" + 0.015*"poder" + 0.015*"deber" + 0.015*"tener" + 0.015*"médico" + 0.015*"analizar" + 0.015*"negociar"'), (3, '0.024*"ser" + 0.019*"trofeo" + 0.019*"tener" + 0.014*"do" + 0.014*"nuevo" + 0.014*"valencia" + 0.014*"parejo" + 0.014*"rey" + 0.014*"jugador" + 0.010*"anterior"'), (4, '0.034*"ser" + 0.023*"planeta" + 0.017*"comer" + 0.012*"marte" + 0.012*"mitología" + 0.012*"pequeño" + 0.012*"formar" + 0.012*"sol" + 0.012*"fruto" + 0.012*"recetar"'), (5, '0.002*"dato" + 0.002*"ser" + 0.002*"tener" + 0.002*"comer" + 0.002*"trofeo" + 0.002*"valencia" + 0.002*"haber" + 0.002*"an

# Resultado gráfico

In [42]:
pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

C:\Users\jcasa\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [43]:
pyLDAvis.display(data)